In [1]:
import os, requests, time
import pandas as pd
import pyodbc
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
os.makedirs(os.path.join("Real Property Dataset"),exist_ok = True)

In [3]:
def Scrape_Data(driver): 
    House_Information = []
    Page = driver.page_source
    Source = BeautifulSoup(Page, "lxml")

    Titles = Source.find_all("span", class_="pr-title js__card-title")
    Areas = Source.find_all("span", class_="re__card-config-area js__card-config-item")
    Bedrooms = Source.find_all("span", class_="re__card-config-bedroom js__card-config-item")
    Bathrooms = Source.find_all("span", class_="re__card-config-toilet js__card-config-item")
    Location = Source.find_all("div", class_="re__card-location")
    Prices = Source.find_all("span", class_="re__card-config-price js__card-config-item")
    
    for title, area, bedroom, bathroom, price, local in zip(Titles, Areas, Bedrooms, Bathrooms, Prices, Location):
        House_Information.append({
            'House Title': title.text.strip(),
            'Area': area.text.strip(),
            'Bedrooms': bedroom.find('span').text.strip(),
            'Bathrooms': bathroom.find('span').text.strip(),
            'Location': local.text.strip(),
            'Price': price.text.strip()
        })
    
    return House_Information

In [4]:

def Browser_Automation():
    driver = webdriver.Edge()
    driver.set_window_size(1280, 720)
    URL = "https://batdongsan.com.vn/ban-nha-rieng-tp-hcm?cIds=325,163"
    driver.get(URL)
    
    House_List = []
    
    def close_popups():
        while True:
            try:
                close_button = WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "span.close")))
                close_button.click()
                time.sleep(0.5)
            except Exception:
                break
            
    while True:
        try:
            close_popups() 
            House_Data = Scrape_Data(driver)
            House_List.extend(House_Data)
            Next_Page = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.re__pagination-icon i.re__icon-chevron-right--sm")))
            Next_Page.click()
            time.sleep(1)                                   
        except Exception:
            break
            
    driver.quit()
    return House_List

Scrape_House_List = pd.DataFrame(Browser_Automation())
Scrape_House_List.to_csv(os.path.join("Real Property Dataset", "Real Property Information.csv"), header=True, index=False)

In [5]:
def Data_Cleaning():
    House_List = pd.read_csv(os.path.join("Real Property Dataset", "Real Property Information.csv"))

    House_List["House Title"] = House_List["House Title"].str.replace('"', "")
    House_List["Area"] = House_List["Area"].str.replace("m²", "")
    House_List["Location"] = House_List["Location"].str.replace("·\n", "")
    House_List["Price"] = House_List["Price"].str.replace("tỷ", "").str.replace(",", ".")

    House_List.to_csv(os.path.join("Real Property Dataset", "Real Property Information.csv"), header=True, index=False)

Data_Cleaning()

In [ ]:
House_List.close()

In [5]:
House_List = pd.read_csv(os.path.join("Real Property Dataset", "Real Property Information.csv"))

Connection = pyodbc.connect("Driver={SQL Server};" 
                            "Server=QUYSEGGS;" 
                            "Database=tempdb;"
                            "Trusted_Connection=yes;")
Column_Name = House_List.columns.tolist()

Cursor = Connection.cursor()

Table_Name = "Real Property Information"

Create_Table = f"create table {Table_Name} ("

for Column in Column_Name:
    if House_List[Column].dtype == 'int64':
        Create_Table += f"{Column} INT, "
    elif House_List[Column].dtype == 'float64':
        Create_Table += f"{Column} FLOAT, "
    else:
        Create_Table += f"{Column} NVARCHAR(255), "
        
# Loại bỏ dấu phẩy cuối cùng và đóng ngoặc
Create_Table = Create_Table.rstrip(", ") + ")"

Cursor.execute(Create_Table)
Connection.commit()

Insert_Cmd = f"INSERT INTO {Table_Name} ({', '.join(Column_Name)}) VALUES ({', '.join(['?' for _ in Column_Name])})"
# Lặp qua từng hàng trong DataFrame và chèn dữ liệu vào bảng
for index, row in House_List.iterrows():
    Cursor.execute(Insert_Cmd, tuple(row))

Connection.commit()

Cursor.close()
Connection.close()

ParserError: Error tokenizing data. C error: Expected 7 fields in line 4, saw 10
